In [3]:
import pytorch_lightning as pl
import os
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
import matplotlib as matp
import numpy as np
from Hang.utils_u_groupnorm_pytorchLightning import *
from utils import *
import array as arr
from torch.utils import data
from numpy import zeros
import time as time
import pdb
import nibabel as nib
import torchvision.transforms
import os
import random
from tqdm import tqdm

torch.cuda.set_device(0)
np.random.seed(0)
random.seed(5)
torch.manual_seed(0)

/home/jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jeremy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [4]:
numbers = ["0001","0017","0018","0038","0040","0042","0046","0087","0090","0108","0116","0131","0178","0190",
           "0227","0248","0267","0282","0285","0398","0448","0466","0504","0514","0535","0564","0598","0606",
           "0607","0618","0620","0623","0642","0646","0655","0668","0675","0681","0719","0761","0762","0783",
           "0786","0868","0877","0887","0895","0902","0931","0979","1007","1013","1029","1033","1068","1142",
           "1143","1163","1190","1260","1275","1347","1383","1389","1416","1435","1441","1447","1451","1514",
           "1520","1602","1611","1621","1680","1684","1686","1710","1720","1739","1743","1749","1753","1760",
           "1795","1805","1845","1858","1876","1889","1892","1898","1899","1918","1924","1932","1952","1961",
           "1972","1987","2003","2007","2016","2020","2022","2030","2045","2047","2049","2053","2055","2074",
           "2077","2080","2091","2094","2103","2115","2128","2142","2144","2146","2152","2156","2158","2160",
           "2161","2179","2180","2181","2183","2186","2188","2212","2221","2231","2234","2245"]
SPLIT = 50
# numbers = numbers[SPLIT:SPLIT + 25]
numbers = numbers[SPLIT + 25:]
mni_numbers = ['2245']
SIM_BRAIN = '2245'

In [5]:
len(numbers)

63

In [6]:
stats = np.load("stats.npy")

In [7]:
stats

array([[7.67116852e+00, 4.64313265e+00],
       [9.06276163e+01, 2.59251717e+01],
       [9.00422508e+00, 8.18314496e+00],
       [8.41492397e+00, 5.14438985e+00],
       [6.72563369e+01, 1.34539774e+01],
       [1.99994947e+03, 1.98949697e-01]])

### Grab 256x256 brains

In [8]:
masks = []
masks_tight = []

for number in numbers:
    file_root = "../" + number + "/"
    masks_tight.append(file_root + "tightmask.nii.gz")
    masks.append(file_root + "brain_mask.nii.gz")

mask_array, mask_array_tight = [], []

for i in range(len(masks)):
    mask_array.append(nib.load(masks[i]).get_fdata()) 
    mask_array_tight.append(nib.load(masks_tight[i]).get_fdata()) 

In [10]:
fname, masks, lesions = [], [], []
w1, w2, w3 = [], [], []
t1, t2, t3 = [], [], []

for number in numbers:
    file_root = "../" + number + "/"
    masks.append(file_root + "tightmask.nii.gz")
    lesions.append(file_root + "lesion.nii.gz")
    fname.append(file_root + "FASTT2_FULL.nii.gz")
    w1.append(file_root + "w1.nii.gz")
    w2.append(file_root + "w2.nii.gz")
    w3.append(file_root + "w3.nii.gz")
    t1.append(file_root + "t1.nii.gz")
    t2.append(file_root + "t2.nii.gz")
    t3.append(file_root + "t3.nii.gz")

mask_array, lesion_mask_array = [], []
brains, labels = [], []

brain2245, label2245 = [], []
mask2245, lesion2245 = [], []

for i in range(len(masks)):
    mask_array.append(nib.load(masks[i]).get_fdata()) 
    if (numbers[i] == SIM_BRAIN):
        mask2245 = nib.load(masks[i]).get_fdata()[None]
    
for i in range(len(lesions)):
    data = nib.load(lesions[i]).get_fdata()
    data[data >= 1] = 1
    lesion_mask_array.append(data) 
    if (numbers[i] == SIM_BRAIN):
        lesion2245 = data[None]

for i in range(len(masks)):
    brain = []
    for j in range(6):
        brain.append(nib.load(fname[i]).get_fdata()[:,:,:,j]  * mask_array[i])
    brains.append(brain)
    if (numbers[i] == SIM_BRAIN):
        brain2245 = np.array(brain)[None]
        
for i in range(len(masks)):
    label = []
    label.append(nib.load(w1[i]).get_fdata() * mask_array[i])
    label.append(nib.load(w2[i]).get_fdata() * mask_array[i])
    label.append(nib.load(w3[i]).get_fdata() * mask_array[i])
    label.append(nib.load(t1[i]).get_fdata() * mask_array[i])
    label.append(nib.load(t2[i]).get_fdata() * mask_array[i])
    label.append(nib.load(t3[i]).get_fdata() * mask_array[i]) 
    labels.append(label)
    if (numbers[i] == SIM_BRAIN):
        label2245 = np.array(label)[None]

labels = np.array(labels)
brains = np.array(brains)
mask_array = np.array(mask_array)
lesion_mask_array = np.array(lesion_mask_array)

### Simulation

In [11]:
def generateBrains(brainRef, labels, masks, noise):
    label = labels[0].copy()
    brains = np.zeros((NUM_BRAINS,)+label.shape)
    echoTimes = [0, 7.5, 17.5, 67.5, 147.5, 307.5]
    for labelIdx in range(len(labels)):
        label = labels[labelIdx].copy()
        for i in range(6):
            brains[labelIdx,i] = label[0] * np.exp(-echoTimes[i] / (label[3] + 1e-16)) + \
                                 label[1] * np.exp(-echoTimes[i] / (label[4] + 1e-16)) + \
                                 label[2] * np.exp(-echoTimes[i] / (label[5] + 1e-16))
            np.nan_to_num(brains[labelIdx,i], copy=False)
        scale = brainRef[labelIdx,0] / (brains[labelIdx,0] + 1e-16)
        for i in range(6):
            brains[labelIdx,i] *= scale
    brains += noise
    for i in range(6):
        brains[:,i] *= masks
    return brains

### Reproducibility

#### Segmentation to mask

In [12]:
reproMaskFiles = []
reproFiles = []
reproMWFFiles = []

reproNumbers = ['125545', '093924', '151026', '111721', '093923', '151025', '113308', '113307', '171732', '101851', 
                '125546', '163835', '163834', '154655', '101850', '175630', '175631', '184052', '111720', '184051']
for number in reproNumbers:
    file_root = "../" + number + "/"
    reproMaskFiles.append(file_root + "tightmask.nii.gz")
    reproFiles.append(file_root + "FASTT2_FULL.nii.gz")
    reproMWFFiles.append(file_root + "MWF.nii.gz")

reproMasks = []
reproBrains = []
reproMWFs = []
for mask in reproMaskFiles:
    mask = nib.load(mask).get_fdata()
    reproMasks.append(mask)

for file_idx in range(len(reproFiles)):
    file = reproFiles[file_idx]
    brain = []
    for j in range(6):
        brain.append(nib.load(file).get_fdata()[:,:,:,j]  * reproMasks[file_idx])
    reproBrains.append(brain)
    
for img in reproMWFFiles:
    img = nib.load(img).get_fdata()
    reproMWFs.append(img[None])

reproMasks = np.array(reproMasks)
reproBrains = np.array(reproBrains)
reproMWFs = np.array(reproMWFs)

### Testing

In [13]:
from Hang.unet3dPersonalGroupNorm_pytorchLightning import unet3d

In [24]:
def calculateResultsErrorTable(state_dicts, stats, testBrains, testLabels, testMasks, numbers, lesions = None, 
                               save=False, brainType = "real", lesion = False):
    featureNumPaths = ["MWF", "IEWF", "CSFF", "MWF_T2", "IEWF_T2", "CSFF_T2"]
    for state_dict in state_dicts:
        model = unet3d(0.01,0.0012, is_deconv = False, weight=50).float()
        weight = state_dict.split("/")[-1].split(".")[0]
        state_dict = torch.load(state_dict, map_location='cpu')
        model.load_state_dict(state_dict["state_dict"])
        model = model.float().cuda()
        model.eval()
        results = {}
        results[0] = []
        results[1] = []
        results[2] = []
        results[3] = []
        results[4] = []
        results[5] = []
        size = testBrains.shape[2:] 
        part = ()
        
        for idx in range(len(testBrains)): 
            number = numbers[idx] if len(testBrains) == len(numbers) else 'average'
            mask = testMasks[idx]
            signal = np.divide(testBrains[idx], testBrains[idx][0] + 1e-16)[None]
            output = model(torch.tensor(signal).float().cuda()).cuda()
            label = testLabels[idx].copy()
            for i in range(len(stats)):
                output[0,i] *= stats[i][1]
                output[0,i] += stats[i][0]
            output = output.detach().cpu().numpy()

            guess_sum = output[0,0]+output[0,1]+output[0,2]+1e-16
            output[0,0] /= guess_sum
            output[0,1] /= guess_sum
            output[0,2] /= guess_sum
            output[0,0] *= 100
            output[0,1] *= 100
            output[0,2] *= 100

            if (label.shape[0] != 1):
                guess_sum_label = label[0] + label[1] + label[2]+1e-16
                label[0] /= guess_sum_label
                label[1] /= guess_sum_label
                label[2] /= guess_sum_label
                label[0] *= 100
                label[1] *= 100
                label[2] *= 100
            
            label = np.nan_to_num(label).astype(float)
            output = np.nan_to_num(output).astype(float)
            output = output[0]
            
            for feature_num in range(min(6, len(label))):
                if (lesion):
                    lesionMask = lesions[idx]
                    if (lesionMask.sum() == 0):
                        continue
                    brain = np.zeros(output[0].shape, dtype=float)
                    label_big = np.zeros(output[0].shape, dtype=float)
                    min_val = -3000
                    max_val = 3000
                    if (feature_num == 0):
                        min_val = 0
                        max_val = 100
                    brain = np.clip(output[feature_num] * mask, min_val, max_val)
                    brain = brain * lesionMask
                    label_big = label[feature_num] * mask
                    label_big =  label_big * lesionMask
                    error = abs(-label_big+brain)
                    error = error[lesionMask == 1]
                    results[feature_num].append(np.mean(error))
                else:
                    brain = np.zeros(output[0].shape, dtype=float)
                    label_big = np.zeros(output[0].shape, dtype=float)
                    min_val = -3000
                    max_val = 3000
                    if (feature_num == 0):
                        min_val = 0
                        max_val = 100
                    brain = np.clip(output[feature_num] * mask, min_val, max_val)
                    label_big =  label[feature_num] * mask
                    error = abs(-label_big+brain)
                    error = error[mask == 1]
                    results[feature_num].append(np.mean(error))
                    if (save):
                        brain = brain
                        label_big = label_big
                        brainT = np.zeros(size)
                        labelT = np.zeros(size)
                        brainT[part] = brain
                        labelT[part] = label_big
                        label_big = labelT
                        brain = brainT
                        base = f"results/0_weight_new/{featureNumPaths[feature_num]}/" #  2222
                        sample = f"../{number}/w1.nii.gz" if number != "average" else ""
                        if (len(number) > 5):
                            sample = f"../{number}/MWF.nii.gz" if number != "average" else ""
                        sample = sample if ("MNI" not in brainType and "mni" not in brainType) else ""
                        save_nii(brain, base+f"{number}_pred_UNET_{brainType}.nii.gz", sample)
                        save_nii(label_big, base+f"{number}_label_UNET_{brainType}.nii.gz", sample)
                        save_nii(brain-label_big, base+f"{number}_err_UNET_{brainType}.nii.gz", sample)
    return results

### Save 256x256 test brains

In [15]:
from scipy.stats import ttest_ind

In [16]:
state_dicts = ["../unetpp_big_gpu0/_ckpt_epoch_56.ckpt"] 

In [13]:
files = [i for i in os.listdir("../unetpp_big_gpu0") if i[0] == '_']
nums = sorted([int(i.split("_")[3].split(".")[0]) for i in files])

In [14]:
files = ['../unetpp_big_gpu0/_ckpt_epoch_' + str(i) + '.ckpt' for i in nums]

In [20]:
res256 = calculateResultsErrorTable(state_dicts, stats, brains, labels, mask_array, numbers, save=True, 
                                    brainType='256real')

In [22]:
print(np.mean(res256[0]), np.std(res256[0])) #MAE for L2

0.7261852746776314 0.25554731522405366


In [25]:
res256Lesion = calculateResultsErrorTable(state_dicts, stats, brains, labels, mask_array, numbers, lesion_mask_array,
                                          lesion = True)

In [26]:
print(np.mean(res256Lesion[0]), np.std(res256Lesion[0])) #MAE for L2

0.742977548145684 0.3848298400963046


### Save 256x256 simulation

#### Simulate brain 1260 in 256x256 space

In [27]:
res256snr200 = []
res256snr200Lesion = []
res256snr100 = []
res256snr100Lesion = []
res256snr50 = []
res256snr50Lesion = []
res256snr25 = []
res256snr25Lesion = []


minSum200 = 10000
minSum100 = 10000
minSum50 = 10000
minSum25 = 10000

for i in tqdm(range(500)):
    '''Generating Brain'''
    NUM_BRAINS = len(brain2245)
    SNR = 200
    roi = load_nii("roi.nii.gz")
    noise = np.random.normal(size=(NUM_BRAINS,) + label2245[0].shape, scale=(brain2245[0,0][roi == 1].mean()/SNR))
    simBrains256_snr200 = generateBrains(brain2245, label2245, mask2245, noise)
    SNR = 100
    noise = np.random.normal(size=(NUM_BRAINS,) + label2245[0].shape, scale=(brain2245[0,0][roi == 1].mean()/SNR))
    simBrains256_snr100 = generateBrains(brain2245, label2245, mask2245, noise)
    SNR = 50
    noise = np.random.normal(size=(NUM_BRAINS,) + label2245[0].shape, scale=(brain2245[0,0][roi == 1].mean()/SNR))
    simBrains256_snr50 = generateBrains(brain2245, label2245, mask2245, noise)
    SNR = 25
    noise = np.random.normal(size=(NUM_BRAINS,) + label2245[0].shape, scale=(brain2245[0,0][roi == 1].mean()/SNR))
    simBrains256_snr25 = generateBrains(brain2245, label2245, mask2245, noise)
    
    '''Testing Brain'''
    tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr200, label2245, mask2245, [SIM_BRAIN], 
                                         save=False, brainType='256sim_snr200')
    tempResLesion = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr200, label2245, mask2245, 
                                               [SIM_BRAIN], lesion2245, save=False, lesion=True)
    res256snr200.append(tempRes)
    res256snr200Lesion.append(tempResLesion)
    if (sum(tempRes) < minSum200):
        minSum200 = sum(tempRes)
        tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr200, label2245, mask2245, [SIM_BRAIN], 
                                             save=True, brainType='256sim_snr200')

    tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr100, label2245, mask2245, [SIM_BRAIN], 
                                         save=False, brainType='256sim_snr100')
    tempResLesion = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr100, label2245, mask2245, 
                                               [SIM_BRAIN], lesion2245, save=False, lesion=True)
    res256snr100.append(tempRes)
    res256snr100Lesion.append(tempResLesion)
    if (sum(tempRes) < minSum100):
        minSum100 = sum(tempRes)
        tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr100, label2245, mask2245, [SIM_BRAIN], 
                                             save=True, brainType='256sim_snr100')
        
    tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr50, label2245, mask2245, [SIM_BRAIN], 
                                         save=False, brainType='256sim_snr50')
    tempResLesion = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr50, label2245, mask2245, 
                                         [SIM_BRAIN], lesion2245,  save=False, lesion=True)
    res256snr50.append(tempRes)
    res256snr50Lesion.append(tempResLesion)
    if (sum(tempRes) < minSum50):
        minSum50 = sum(tempRes)
        tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr50, label2245, mask2245, [SIM_BRAIN], 
                                             save=True, brainType='256sim_snr50')
        
    tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr25, label2245, mask2245, [SIM_BRAIN], 
                                         save=False, brainType='256sim_snr25')
    tempResLesion = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr25, label2245, mask2245, 
                                         [SIM_BRAIN], lesion2245,  save=False, lesion=True)
    res256snr25.append(tempRes)
    res256snr25Lesion.append(tempResLesion)
    if (sum(tempRes) < minSum25):
        minSum25 = sum(tempRes)
        tempRes = calculateResultsErrorTable(state_dicts, stats, simBrains256_snr25, label2245, mask2245, [SIM_BRAIN], 
                                             save=True, brainType='256sim_snr25')

100%|██████████| 500/500 [1:38:06<00:00, 11.77s/it]


In [29]:
res256snr200Avg = compressResults(res256snr200)
res256snr100Avg = compressResults(res256snr100)
res256snr50Avg = compressResults(res256snr50)
res256snr25Avg = compressResults(res256snr25)

res256snr200LesionAvg = compressResults(res256snr200Lesion)
res256snr100LesionAvg = compressResults(res256snr100Lesion)
res256snr50LesionAvg = compressResults(res256snr50Lesion)
res256snr25LesionAvg = compressResults(res256snr25Lesion)

In [31]:
simData = [res256snr200, res256snr100, res256snr50, res256snr25]
simDataLesion = [res256snr200Lesion, res256snr100Lesion, res256snr50Lesion, res256snr25Lesion]

### Save Repro

In [32]:
resRepro = calculateResultsErrorTable(state_dicts, stats, reproBrains, reproMWFs, reproMasks, reproNumbers, 
                                      save=True, brainType='repro')

### Saving Results

In [34]:
np.save("unetData.npy", [res256, res256Lesion, 
                        res256snr200Avg, res256snr100Avg, res256snr50Avg, res256snr25Avg,
                        res256snr200LesionAvg, res256snr100LesionAvg, res256snr50LesionAvg, res256snr25LesionAvg,
                        resRepro, simData, simDataLesion], allow_pickle=True)

/home/jeremy/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
